In [1]:
import joblib
import geopandas as gpd
from pystac.client import Client
from odc.stac import load
import pandas as pd
import rasterio
import numpy as np
import xarray as xr
from rasterio.transform import from_origin
from sklearn.ensemble import RandomForestClassifier
from utils import calculate_band_indices, apply_masks, scale

In [2]:
version = "test_21052025"

In [3]:
# Load the model
model = joblib.load("models/model-geomad-joined-data-11042025.model")

In [4]:
# Open test data
region = gpd.read_file("testing-data/utanglang_postcard.geojson")
region.explore()

In [5]:
bbox = list(region.total_bounds)
year = "2024"

client = Client.open("https://stac.digitalearthpacific.org")
collection = "dep_s2_geomad"

items = client.search(collections=collection, bbox=bbox, datetime=year).item_collection()

print(f"Found {len(items)} items")

Found 1 items


In [6]:
data = load(
    items,
    bbox=bbox,
    measurements=[
        "nir",
        "red",
        "blue",
        "green",
        "emad",
        "smad",
        "bcmad",
        "count",
        "green",
        "nir08",
        "nir09",
        "swir16",
        "swir22",
        "coastal",
        "rededge1",
        "rededge2",
        "rededge3",
    ],
    chunks={"x": 2048, "y": 2048},
)

data

<xarray.Dataset> Size: 348kB
Dimensions:      (y: 147, x: 62, time: 1)
Coordinates:
  * y            (y) float64 1kB -1.957e+06 -1.957e+06 ... -1.958e+06 -1.958e+06
  * x            (x) float64 496B 2.04e+06 2.04e+06 ... 2.04e+06 2.04e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    red          (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    blue         (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    green        (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    emad         (time, y, x) float32 36kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    smad         (time, y, x) float32 36kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    swir22       (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    coastal      (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    rededge1     (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    rededge2     (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    rededge3     (time, y, x) uint16 18kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>

In [7]:
masked = calculate_band_indices(apply_masks(scale(data)))
masked

<xarray.Dataset> Size: 2MB
Dimensions:      (time: 1, y: 147, x: 62)
Coordinates:
  * y            (y) float64 1kB -1.957e+06 -1.957e+06 ... -1.958e+06 -1.958e+06
  * x            (x) float64 496B 2.04e+06 2.04e+06 ... 2.04e+06 2.04e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/27)
    nir          (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    red          (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    blue         (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    green        (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    emad         (time, y, x) float32 36kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    smad         (time, y, x) float32 36kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    ...           ...
    b_g          (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    b_r          (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    mci          (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    ndci         (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    stumpf       (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>
    ln_bg        (time, y, x) float64 73kB dask.array<chunksize=(1, 147, 62), meta=np.ndarray>

In [8]:
# First transform the training points to the same CRS as the data
test = region.to_crs(data.odc.geobox.crs)

# Next get the X and Y values out of the point geometries
training_da = test.assign(x=test.geometry.x, y=test.geometry.y).to_xarray()

# Now we can use the x and y values (lon, lat) to extract values from the median composite
training_values = (
    masked.sel(training_da[["x", "y"]], method="nearest").squeeze().compute().to_pandas()
)

# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([test["cc_id"], training_values], axis=1)
training_array = training_array.drop(
    columns=[
        "y",
        "x",
        "spatial_ref",
        "stumpf",
        "ln_bg",
        "time",
    ]
)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

/srv/conda/envs/notebook/lib/python3.10/site-packages/dask/core.py:127: RuntimeWarning: divide by zero encountered in log
  return func(*(_execute_task(a, cache) for a in args))


,cc_id,nir,red,blue,green,emad,smad,bcmad,count,nir08,...,rededge3,mndwi,ndti,cai,ndvi,ndwi,b_g,b_r,mci,ndci
0,6,0.0239,0.0302,0.0772,0.0705,0.086266,0.000006,0.000037,0.0055,0.0263,...,0.0272,0.447639,-0.400199,-0.024552,-0.116451,0.493644,1.095035,2.556291,0.796667,-0.003322
1,6,0.0271,0.0334,0.0799,0.0742,0.083707,0.000005,0.000033,0.0057,0.0284,...,0.0299,0.454902,-0.379182,-0.014603,-0.104132,0.464956,1.076819,2.392216,0.833846,-0.013657
2,6,0.0289,0.0347,0.0830,0.0774,0.087613,0.000005,0.000034,0.0061,0.0303,...,0.0317,0.444030,-0.380910,-0.040100,-0.091195,0.456256,1.072351,2.391931,0.840116,-0.004342
3,6,0.0254,0.0338,0.0764,0.0739,0.085507,0.000004,0.000027,0.0059,0.0276,...,0.0284,0.469185,-0.372331,0.042006,-0.141892,0.488419,1.033829,2.260355,0.751479,0.000000
4,6,0.0249,0.0314,0.0727,0.0671,0.072944,0.000005,0.000031,0.0060,0.0258,...,0.0274,0.453954,-0.362437,0.011557,-0.115453,0.458696,1.083458,2.315287,0.792994,0.000000


In [9]:
training_array.columns

Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
       'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'ndwi', 'b_g',
       'b_r', 'mci', 'ndci'],
      dtype='object')

In [10]:
# # Run a prediction and compare with our known values
# coastal_class = training_array["cc_id"]
# variables = training_array.drop(columns=["cc_id", "time", 'stumpf', 'ln_bg'])
# training_array = training_array.drop(columns=["time", 'stumpf', 'ln_bg'])
# variables

In [11]:
masked = masked.drop_vars(['stumpf', 'ln_bg'])

In [12]:
# The classes are the first column
classes = np.array(training_array)[:, 0]

# The observation data is everything after the second column
observations = np.array(training_array)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [13]:
stacked_arrays = masked.squeeze()#.stack(dims=["y", "x"])#.transpose()
stacked_arrays = stacked_arrays.to_dataarray()

In [14]:
stacked_arrays_2d = stacked_arrays.stack(new_dim=("y", "x")) 
reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')
stacked_arrays_2d.shape


(25, 9114)

In [15]:
# Replace any infinities with NaN
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("inf"))
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("-inf"))

# Replace any NaN values with 0
df = stacked_arrays_2d.squeeze().fillna(0).transpose().to_pandas()

# Remove the all-zero rows
zero_mask = (df == 0).all(axis=1)  # Creates a boolean Series
non_zero_df = df.loc[~zero_mask]  # Filters out all-zero rows

# Create a new array to hold the predictions
full_pred = pd.Series(np.nan, index=df.index)

reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')

In [16]:
feature_names = list(training_array.columns[1:])  # skip class column
print(feature_names)

['nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad', 'count', 'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'ndwi', 'b_g', 'b_r', 'mci', 'ndci']


In [17]:

predicted = model.predict(reordered_data_array)


# # Reshape back to the original 2D array
reordered_data_array = predicted.reshape(len(masked.y), len(masked.x))

# # Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    reordered_data_array, coords={"y": masked.y, "x": masked.x}, dims=["y", "x"]
)

In [18]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

# Check for NaN values
if np.isnan(predicted_da).any():
    print("NaN values found in the data")
    # Handle NaN values, e.g. by filling them
    predicted_da = predicted_da.fillna(0)  # Replace NaN with 0 or appropriate value

object


In [19]:
feature_names = list(training_array.columns[1:])  # skip class column
print(len(feature_names))

25


In [20]:
features_for_prediction = masked[feature_names]# Convert to shape (n_samples, n_features)
# X = features_for_prediction.to_array().transpose('y', 'x', 'variable').values  # (height, width, n_features)
# height, width, n_features = X.shape
# X_flat = X.reshape(-1, n_features)

In [21]:
# (variable, time, y, x) -> (y, x, variable)
X = features_for_prediction.to_array().transpose('y', 'x', 'variable', 'time').values  # (y, x, variable, time)
# Remove the singleton 'time' dimension (if time=1)
X = np.squeeze(X, axis=-1)  # now X.shape is (y, x, variable)

In [22]:
X = features_for_prediction.to_array().transpose('y', 'x', 'variable', 'time').values
X = np.squeeze(X, axis=-1)

In [23]:
X = features_for_prediction.to_array().transpose('y', 'x', 'variable', ...).values
# ... will preserve any other dimensions, like time
X = np.squeeze(X)  # Squeeze out singleton dimensions

In [24]:
# Select the first (or only) time index
X = features_for_prediction.isel(time=0).to_array().transpose('y', 'x', 'variable').values

In [25]:
X = features_for_prediction.isel(time=0).to_array().transpose('y', 'x', 'variable').values  # (y, x, n_features)
X_flat = X.reshape(-1, X.shape[-1])

In [26]:
reordered_data_array

array([['6', '6', '6', ..., '2', '2', '2'],
       ['6', '6', '6', ..., '2', '2', '2'],
       ['6', '6', '6', ..., '2', '2', '2'],
       ...,
       ['2', '2', '6', ..., '2', '2', '2'],
       ['2', '6', '2', ..., '2', '2', '2'],
       ['2', '2', '6', ..., '2', '2', '2']], dtype=object)

In [27]:
# Predict the classes

predicted = model.predict(X_flat)

In [28]:
predicted

array(['6', '6', '6', ..., '2', '2', '2'], dtype=object)

In [29]:
predictions = model.predict(masked)

TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_dataarray()` method.

In [ ]:
predictions

In [58]:
predictions.odc.write_cog(f":predictions_{version}.tif")

AttributeError: 'numpy.ndarray' object has no attribute 'odc'

In [55]:
# Reshape
height = masked.sizes['y']
width = masked.sizes['x']
pred_2d = predictions.reshape(height, width)

# Define transform and CRS (set these appropriately for your data)


transform = from_origin(west=0, north=0, xsize=1, ysize=1)  # Example values
crs = 'EPSG:4326'  # Example CRS


with rasterio.open(
    f"predictions_{version}.tif", 'w',
    driver='COG',  # or 'GTiff' if COG fails
    height=height,
    width=width,
    count=1,
    dtype=pred_2d.dtype,
    crs=masked.odc.geobox.crs,
    transform=masked.odc.geobox.transform,
) as dst:
    dst.write(pred_2d, 1)

NameError: name 'predictions' is not defined